In [1]:
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from mpl_toolkits.basemap import Basemap
from numpy.fft import fft, fftfreq, ifft
import pandas as pd

from scipy.fftpack import *
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

#from scipy import signal
#from eofs.standard import Eof

plt.rcParams['font.family'] = 'deJavu Sans'
plt.rcParams['font.size'] = 35 #22
plt.rcParams['font.weight'] = 'bold' #22
#plt.rcParams['figure.figsize'] = [35,10]
import matplotlib.ticker as ticker

In [2]:
pc_1=pd.read_excel('C:/Users/User/Desktop/MATERI KULIAH GANJIL 2024/BERPIKIR KOMPUTASI/PERTEMUAN 6 (BELAJAR LOOP)\MENYAMAKAN KOORDINAT/data_curah_hujan.xlsx')
pc_1_nc=pc_1[0:480] #48-16
#northern=northern_bmkg[120:480] #90-19
pc_1

,years,tahun,bulan,tanggal,pc1,pc2,pc3
0,1983-01-01,1983,1,1,14.387097,3.103226,6.774194
1,1983-02-01,1983,2,1,15.000000,4.960714,8.642857
2,1983-03-01,1983,3,1,7.677419,10.825806,1.774194
3,1983-04-01,1983,4,1,6.566667,4.670000,2.266667
4,1983-05-01,1983,5,1,1.612903,1.374194,3.838710
...,...,...,...,...,...,...,...
475,2022-08-01 00:00:00,2022,8,1,0.025806,0.145161,0.119355
476,2022-09-01 00:00:00,2022,9,1,0.526667,0.500000,0.356667
477,2022-10-01 00:00:00,2022,10,1,0.851613,2.180645,4.464516
478,2022-11-01 00:00:00,2022,11,1,10.923333,3.880000,8.176667


In [3]:
ds_sst_slp_wind_era5=xr.open_dataset("D:/ANALISIS RR/S3/FULL_LONG-1940-2022-ver1-5-XX.nc")

t_start='1983-01-01'
t_stop='2022-12-01'

min_lon = 40 
max_lon = 250 
min_lat = -35 
max_lat = 35

#ds_uwind_sel = ds_uwind.sel(time=slice(t_start, t_stop), lat=slice(max_lat,min_lat), lon=slice(min_lon,max_lon))
#ds_vwind_sel = ds_vwind.sel(time=slice(t_start, t_stop), lat=slice(max_lat,min_lat), lon=slice(min_lon,max_lon))
#ds_slp_sel = ds_slp.sel(time=slice(t_start, t_stop), lat=slice(max_lat,min_lat), lon=slice(min_lon,max_lon))
ds_era5_sel = ds_sst_slp_wind_era5.sel(time=slice(t_start, t_stop), latitude=slice(max_lat,min_lat), longitude=slice(min_lon,max_lon))
#ds_era5_sel

ds_era5_sel_climo = ds_era5_sel.groupby('time.month').mean('time')
ds_era5_sel_anoms = (ds_era5_sel.groupby('time.month') - ds_era5_sel_climo)

In [68]:
ds_era5_sel

<xarray.Dataset>
Dimensions:    (time: 828, latitude: 281, longitude: 841)
Coordinates:
  * longitude  (longitude) float32 40.0 40.25 40.5 40.75 ... 249.5 249.8 250.0
  * latitude   (latitude) float32 35.0 34.75 34.5 34.25 ... -34.5 -34.75 -35.0
  * time       (time) datetime64[ns] 1948-01-01 1948-02-01 ... 2016-12-01
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
    sst        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-03-10 06:55:34 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [9]:
rr_fcor=[]
list_tp=pc_1.pc1.to_list()
for k in range(len(list_tp)):
    rr_fcor.append([[list_tp[k] for i in range(len(ds_era5_sel.longitude))] for j in range(len(ds_era5_sel.latitude))])
    
print(len(rr_fcor), len(rr_fcor[0]), len(rr_fcor[0][0]))

480 281 841


In [73]:
#rr_fcor

In [10]:
rr_fcorx=[[list_tp for i in range(len(ds_era5_sel.latitude))] for j in range(len(ds_era5_sel.longitude))]
print(len(rr_fcorx), len(rr_fcorx[0]), len(rr_fcorx[0][0]))

841 281 480


In [11]:
for ii in range(2):
    for jj in range(3):
#        for kk in range(len(northern.tp)):
        rr_fcorx[jj][ii]=(pc_1.pc1.to_list())

In [12]:
print(len(rr_fcorx), len(rr_fcorx[0]), len(rr_fcorx[0][0]))

841 281 480


In [13]:
rr_fcorxx=[[np.nan for i in range(2)] for j in range(3)] 
for ii in range(2):
    for jj in range(3):
#        for kk in range(len(northern.tp)):
        rr_fcorxx[jj][ii]=(pc_1.pc1.to_list())

In [14]:
print(len(rr_fcorxx), len(rr_fcorxx[0]), len(rr_fcorxx[0][0]))

3 2 480


In [79]:
#print(len(northern.years), len(ds_era5_sel.longitude), len(ds_era5_sel.latitude))

In [80]:
#len(ds_era5_sel.longitude)

In [15]:
for i in range(len(ds_era5_sel.latitude)):
    for j in range(len(ds_era5_sel.longitude)):
        #for k in range(len(northern.tp)):
        rr_fcorxx[jj][ii]=(pc_1.pc1.to_list())

In [16]:
print(len(rr_fcor), len(rr_fcor[0]), len(rr_fcor[0][0]))

480 281 841


In [17]:
sst_rr_xar=xr.DataArray(
        data = rr_fcor,
        dims = ['time','latitude','longitude'],
        coords = {'time':pd.to_datetime(pc_1.years),'latitude': ds_era5_sel.latitude,'longitude': ds_era5_sel.longitude},
        attrs = {'rr': 'rr_stasiun data', 'units': 'mm'}
        )

In [18]:
sst_rr_xar

<xarray.DataArray (time: 480, latitude: 281, longitude: 841)>
array([[[14.38709677, 14.38709677, 14.38709677, ..., 14.38709677,
         14.38709677, 14.38709677],
        [14.38709677, 14.38709677, 14.38709677, ..., 14.38709677,
         14.38709677, 14.38709677],
        [14.38709677, 14.38709677, 14.38709677, ..., 14.38709677,
         14.38709677, 14.38709677],
        ...,
        [14.38709677, 14.38709677, 14.38709677, ..., 14.38709677,
         14.38709677, 14.38709677],
        [14.38709677, 14.38709677, 14.38709677, ..., 14.38709677,
         14.38709677, 14.38709677],
        [14.38709677, 14.38709677, 14.38709677, ..., 14.38709677,
         14.38709677, 14.38709677]],

       [[15.        , 15.        , 15.        , ..., 15.        ,
         15.        , 15.        ],
        [15.        , 15.        , 15.        , ..., 15.        ,
         15.        , 15.        ],
        [15.        , 15.        , 15.        , ..., 15.        ,
         15.        , 15.        ],
...
        [10.92333333, 10.92333333, 10.92333333, ..., 10.92333333,
         10.92333333, 10.92333333],
        [10.92333333, 10.92333333, 10.92333333, ..., 10.92333333,
         10.92333333, 10.92333333],
        [10.92333333, 10.92333333, 10.92333333, ..., 10.92333333,
         10.92333333, 10.92333333]],

       [[ 9.99677419,  9.99677419,  9.99677419, ...,  9.99677419,
          9.99677419,  9.99677419],
        [ 9.99677419,  9.99677419,  9.99677419, ...,  9.99677419,
          9.99677419,  9.99677419],
        [ 9.99677419,  9.99677419,  9.99677419, ...,  9.99677419,
          9.99677419,  9.99677419],
        ...,
        [ 9.99677419,  9.99677419,  9.99677419, ...,  9.99677419,
          9.99677419,  9.99677419],
        [ 9.99677419,  9.99677419,  9.99677419, ...,  9.99677419,
          9.99677419,  9.99677419],
        [ 9.99677419,  9.99677419,  9.99677419, ...,  9.99677419,
          9.99677419,  9.99677419]]])
Coordinates:
  * time       (time) datetime64[ns] 1983-01-01 1983-02-01 ... 2022-12-01
  * latitude   (latitude) float32 35.0 34.75 34.5 34.25 ... -34.5 -34.75 -35.0
  * longitude  (longitude) float32 40.0 40.25 40.5 40.75 ... 249.5 249.8 250.0
Attributes:
    rr:       rr_stasiun data
    units:    mm

In [19]:
ds = xr.Dataset({
    'rr_spasial': xr.DataArray(
            data = rr_fcor,
            dims = ['time','latitude','longitude'],
            coords = {'time':pd.to_datetime(pc_1.years),'latitude': ds_era5_sel.latitude,'longitude': ds_era5_sel.longitude},
            attrs = {'rr': 'rr_stasiun data', 'units': 'mm'}
            )
            },
        attrs = {'RR': 'Data Rata Rata Stasiun'}
    )

In [20]:
ds

<xarray.Dataset>
Dimensions:     (time: 480, latitude: 281, longitude: 841)
Coordinates:
  * time        (time) datetime64[ns] 1983-01-01 1983-02-01 ... 2022-12-01
  * latitude    (latitude) float32 35.0 34.75 34.5 34.25 ... -34.5 -34.75 -35.0
  * longitude   (longitude) float32 40.0 40.25 40.5 40.75 ... 249.5 249.8 250.0
Data variables:
    rr_spasial  (time, latitude, longitude) float64 14.39 14.39 ... 9.997 9.997
Attributes:
    RR:       Data Rata Rata Stasiun

In [21]:
ds.to_netcdf("D:/analisis rr/Atika_PC1.nc")